In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import numpy as np
import cv2

In [9]:
bf = cv2.BFMatcher()

img_1 = cv2.imread('./outputs/output01-04.png')
img1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)

img_2 = cv2.imread('./outputs/output05-08.png')
img2 = cv2.cvtColor(img_2,cv2.COLOR_BGR2GRAY)

#sift = cv2.xfeatures2d.SIFT_create()
 
orb = cv2.ORB_create(
    nfeatures=10000,
    scaleFactor=1.2,
    nlevels=8,
    edgeThreshold=31,
    firstLevel=0,
    WTA_K=2,
    scoreType=cv2.ORB_HARRIS_SCORE,
    patchSize=31,
    fastThreshold=20,
)

kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)
matches = bf.knnMatch(des1,des2, k=2)

good = []
for m in matches:
    if (m[0].distance < 0.7*m[1].distance):
        good.append(m)
matches = np.asarray(good)

if (len(matches[:,0]) >= 5):
    src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
    dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
    H, masked = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

    dst = cv2.warpPerspective(img_1,H,((img_1.shape[1] + img_2.shape[1]), img_2.shape[0])) #wraped image
    dst[0:img_2.shape[0], 0:img_2.shape[1]] = img_2 #stitched image
    
    dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(dst_gray, 0, 255, cv2.THRESH_OTSU)
    
    bbox = cv2.boundingRect(thresholded)
    x,y,w,h = bbox
    
    dst = dst[y:y+h, x:x+w]

    cv2.imwrite('output01-08.png',dst)